# Lecture 37: GoogLeNet

In [ ]:
%matplotlib inline
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torchvision import transforms,datasets, models
import torch.optim as optim
from torch.autograd import Variable
import matplotlib.pyplot as plt
import torchvision

import copy
import time

## Load Data:

In [ ]:
# inception_v3 in pytorch requries input to be of size(3x299x299)
apply_transform = transforms.Compose([transforms.Scale(299),transforms.ToTensor()])
BatchSize = 32

trainset = datasets.CIFAR10(root='./CIFAR10', train=True, download=True, transform=apply_transform)
trainLoader = torch.utils.data.DataLoader(trainset, batch_size=BatchSize,
                                          shuffle=True, num_workers=4) # Creating dataloader

testset = datasets.CIFAR10(root='./CIFAR10', train=False, download=True, transform=apply_transform)
testLoader = torch.utils.data.DataLoader(testset, batch_size=BatchSize,
                                         shuffle=False, num_workers=4) # Creating dataloader

In [ ]:
# Size of train and test datasets
print('No. of samples in train set: '+str(len(trainLoader.dataset)))
print('No. of samples in test set: '+str(len(testLoader.dataset)))

## Define network architecture
Inceptionv3 [architecture](https://hackathonprojects.files.wordpress.com/2016/09/v3.png)

In [ ]:
net = models.inception_v3()
print(net)

In [ ]:
# Counting number of trainable parameters
totalParams = 0
for params in net.parameters():
    print params.size()
    totalParams += np.sum(np.prod(params.size()))
print('Total number of parameters: '+str(totalParams))

In [ ]:
# Modifying the last fully-connected layers(including aux network) for 10 classes
net.AuxLogits.fc = nn.Linear(768,10)
net.fc = nn.Linear(2048,10)

In [ ]:
# Copying initial weights for visualization
init_weightConv1 = copy.deepcopy(net.Conv2d_1a_3x3.conv.weight.data) # 1st conv layer
init_weightConv2 = copy.deepcopy(net.Conv2d_2a_3x3.conv.weight.data) # 2nd conv layer

In [ ]:
# Check availability of GPU
use_gpu = torch.cuda.is_available()
if use_gpu:
    print('GPU is available!')   
    net = net.cuda()

## Define loss function and optimizer

In [ ]:
criterion = nn.NLLLoss() # Negative Log-likelihood
optimizer = optim.Adam(net.parameters(), lr=1e-4) # Adam

## Train the network

In [ ]:
iterations = 10
trainLoss = [] # List for saving main loss per epoch
trainAuxLoss = [] # List for saving auxillary loss per epoch
trainTotalLoss = [] # List for saving total loss per epoch
trainAcc = [] # List for saving training accuracy per epoch

testLoss = [] # List for saving testing loss per epoc
testAcc = [] # List for saving testing accuracy per epoch

start = time.time()
for epoch in range(iterations):
    epochStart = time.time()
    runningLoss = 0.0
    runningAuxLoss = 0.0 
    runningTotalLoss = 0.0
    avgAuxLoss = 0.0
    avgTotalLoss = 0.0
    running_correct = 0
    net.train(True) # For training
    for data in trainLoader:
        # Initialize gradients to zero
        optimizer.zero_grad()
        
        inputs,labels = data
        # Wrap them in Variable
        if use_gpu:
            inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())
            # Feed-forward input data through the network        
            outputs,aux_outputs = net(inputs)  
            _, predicted = torch.max(outputs.data, 1)            
            running_correct += (predicted.cpu() == labels.data.cpu()).sum()
        else:
            inputs, labels = Variable(inputs), Variable(labels)  
            # Feed-forward input data through the network        
            outputs,aux_outputs = net(inputs)  
            _, predicted = torch.max(outputs.data, 1)
            running_correct += (predicted == labels.data).sum()             
              
        # Compute loss/error
        loss = criterion(F.log_softmax(outputs), labels)
        aux_loss = criterion(F.log_softmax(aux_outputs), labels)
        total_loss = loss+aux_loss
        # Backpropagate loss and compute gradients
        total_loss.backward()
        # Update the network parameters
        optimizer.step()
        # Accumulate loss per batch
        runningLoss += loss.data[0]    
        runningAuxLoss += aux_loss.data[0]    
        runningTotalLoss += total_loss.data[0] 
        
    avgTrainAcc = running_correct/50000.0
    avgTrainLoss = runningLoss/50000.0
    avgAuxLoss = runningAuxLoss/50000.0
    avgTotalLoss = runningTotalLoss/50000.0
    trainAcc.append(avgTrainAcc)
    trainLoss.append(avgTrainLoss)
    trainAuxLoss.append(avgAuxLoss)
    trainTotalLoss.append(avgTotalLoss)
    
    # Evaluating performance on test set for each epoch
    net.train(False) # For testing [Affects batch-norm and dropout layers (if any)]
    running_correct = 0
    for data in testLoader:
        inputs,labels = data
        # Wrap them in Variable
        if use_gpu:
            inputs, labels= Variable(inputs.cuda()), Variable(labels.cuda())
            outputs = net(inputs)
            _, predicted = torch.max(outputs.data, 1)            
            running_correct += (predicted.cpu() == labels.data.cpu()).sum()
        else:
            inputs, labels = Variable(inputs), Variable(labels)
            outputs = net(inputs)
            _, predicted = torch.max(outputs.data, 1)
            running_correct += (predicted == labels.data).sum()
        
        loss = criterion(F.log_softmax(outputs), labels)
        runningLoss += loss.data[0] 
        
    avgTestLoss = runningLoss/10000.0
    avgTestAcc = running_correct/10000.0
    testLoss.append(avgTestLoss)
    testAcc.append(avgTestAcc)
        
    # Plotting training loss vs aux_loss
    fig1 = plt.figure(1)        
    plt.plot(range(epoch+1),trainLoss,'r-',label='main loss')  
    plt.plot(range(epoch+1),trainAuxLoss,'g-',label='aux') 
    if epoch==0:
        plt.legend(loc='upper left')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')   
    
    # Plotting training loss vs Epochs
    fig2 = plt.figure(2)        
    plt.plot(range(epoch+1),trainTotalLoss,'r-',label='train')  
    plt.plot(range(epoch+1),testLoss,'g-',label='test') 
    if epoch==0:
        plt.legend(loc='upper left')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')   
    # Plotting testing accuracy vs Epochs
    fig3 = plt.figure(3)        
    plt.plot(range(epoch+1),trainAcc,'r-',label='train')    
    plt.plot(range(epoch+1),testAcc,'g-',label='test')        
    if epoch==0:
        plt.legend(loc='upper left')
        plt.xlabel('Epochs')
        plt.ylabel('Accuracy')    
    epochEnd = time.time()-epochStart
    print('Iteration: {:.0f} /{:.0f}  ;  Training Loss: {:.6f} ; Testing Acc: {:.3f} ; Time consumed: {:.0f}m {:.0f}s '\
          .format(epoch + 1,iterations,avgTrainLoss,avgTestAcc*100,epochEnd//60,epochEnd%60))
end = time.time()-start
print('Training completed in {:.0f}m {:.0f}s'.format(end//60,end%60))


In [ ]:
# Copying trained weights for visualization
if use_gpu:
    trained_weightConv1 = copy.deepcopy(net.Conv2d_1a_3x3.conv.weight.data.cpu())
    trained_weightConv2 = copy.deepcopy(net.Conv2d_2a_3x3.conv.weight.data.cpu())
else:
    trained_weightConv1 = copy.deepcopy(net.Conv2d_1a_3x3.conv.weight.data)
    trained_weightConv2 = copy.deepcopy(net.Conv2d_2a_3x3.conv.weight.data)

## Visualization of weights

In [ ]:
# functions to show an image
def imshow(img, strlabel):
    npimg = img.numpy()
    npimg = np.abs(npimg)
    fig_size = plt.rcParams["figure.figsize"]
    fig_size[0] = 10
    fig_size[1] = 10
    plt.rcParams["figure.figsize"] = fig_size
    plt.figure()
    plt.title(strlabel)
    plt.imshow(np.transpose(npimg, (1, 2, 0)))

In [ ]:
# Visualizing weights of 1st convolutional layer
imshow(torchvision.utils.make_grid(init_weightConv1,nrow=8,normalize=True),'Initial weights: conv1')
imshow(torchvision.utils.make_grid(trained_weightConv1,nrow=8,normalize=True),'Trained weights: conv1')
imshow(torchvision.utils.make_grid(init_weightConv1-trained_weightConv1,nrow=8,normalize=True),'Difference of weights: conv1')

In [ ]:
# Visualizing weights of 2nd convolutional layer
imshow(torchvision.utils.make_grid(init_weightConv2[0].unsqueeze(1),nrow=8,normalize=True),'Initial weights: conv2')
imshow(torchvision.utils.make_grid(trained_weightConv2[0].unsqueeze(1),nrow=8,normalize=True),'Trained weights: conv2')
imshow(torchvision.utils.make_grid(init_weightConv2[0].unsqueeze(1)-trained_weightConv2[0].unsqueeze(1),nrow=8,normalize=True),'Difference of weights: conv2')